In [1]:
from train import train
from inference import inference
from save_mask import save_mask
import csv, os
from utils.module import write_to_csv

In [2]:
!nvidia-smi

Sun Oct 22 02:19:59 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla M60                      On  | 00000000:00:1B.0 Off |                    0 |
| N/A   32C    P8              14W / 150W |      0MiB /  7680MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
project = "231023"
folnames = [project+"_iter1", project+"_iter2", project+"_iter3", project+"_iter4", project+"_iter5"]
os.makedirs('results/'+ project +'_results', exist_ok=True)
csv_filename = 'results/'+project+'_results/results.csv'
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Iteration', 'F1-Score', 'Accuracy', 'Specificity', 'Recall', 'Precision'])

for i in range(len(folnames)):
    if i==0:
        # train(former_folname="hoge", folname=folnames[i], first=True, net="deeplab", epochs=300, batch_size=64)
        scores = inference(former_folname="hoge", folname=folnames[i], net="deeplab", batch_size=64)
        write_to_csv(i+1, scores, csv_filename)
        save_mask(former_folname="hoge", folname=folnames[i], net="deeplab", batch_size=64)
        pass
    else:
        train(former_folname=folnames[i-1], folname=folnames[i], first=False, net="deeplab", epochs=300, batch_size=64, alpha=1000)
        scores = inference(former_folname=folnames[i-1], folname=folnames[i], net="deeplab", batch_size=64)
        write_to_csv(i+1, scores, csv_filename)
        save_mask(former_folname=folnames[i-1], folname=folnames[i], net="deeplab", batch_size=64)

100%|██████████| 16/16 [02:04<00:00,  7.81s/it]


F1-Score : 0.5732582807540894
Accuracy : 0.9970160722732544
Specificity : 0.9983611106872559
Recall : 0.6358425617218018
Precision : 0.5734639763832092


100%|██████████| 4292/4292 [01:47<00:00, 39.76it/s]


Epoch 1/300


100%|██████████| 18/18 [00:05<00:00,  3.25it/s]


...

Train loss: 0.3523223045372194
Train consistency loss: 0.08892247414411004
Validation loss: 0.4355974081489775
1m 14s
Consistency Loss declined to 0.08892247414411004
Early Stopping Counter = 0
Epoch 2/300


100%|██████████| 18/18 [00:05<00:00,  3.34it/s]


Train loss: 0.2637090814690436
Train consistency loss: 0.08107309816266174
Validation loss: 0.46301309764385223
1m 13s
Consistency Loss declined to 0.08107309816266174
Early Stopping Counter = 0
Epoch 3/300


 57%|█████▋    | 88/155 [00:27<00:20,  3.20it/s]


KeyboardInterrupt: 

In [6]:
meanlst = os.listdir("data/unlabeled_mask/231021_iter1/pred_mean_corrected/")
varlst = os.listdir("data/unlabeled_mask/231021_iter1/pred_var/")
print(len(meanlst))
print(len(varlst))

4292
4292


In [3]:
import numpy as np
from PIL import Image
meanlst = os.listdir("data/unlabeled_mask/231021_iter1/pred_mean_corrected/")
varlst = os.listdir("data/unlabeled_mask/231021_iter1/pred_var/")

# # 各画像を処理
for file_name in meanlst:
    file_path = os.path.join("data/unlabeled_mask/231021_iter1/pred_mean_corrected", file_name)
    
    # 画像をnumpy配列に読み込む
    img = np.array(Image.open(file_path))
    
    # 画像サイズが256x256でない場合、警告を表示
    if img.shape != (256, 256):
        print(f"異なるサイズの画像: {file_name}, サイズ: {img.shape}")

In [4]:
import os
import torch

# フォルダのパス
folder_path = "data/unlabeled_mask/231021_iter1/pred_var/"

# フォルダ内の全ての.ptファイルをリストアップ
pt_files = [f for f in os.listdir(folder_path) if f.endswith(".pt")]

# 各ファイルを処理
for file_name in pt_files:
    file_path = os.path.join(folder_path, file_name)
    
    # PyTorch tensorを読み込む
    tensor = torch.load(file_path)
    
    # 配列サイズが(256, 256)でない場合、警告を表示
    if tensor.size() != torch.Size([256, 256]):
        print(f"異なるサイズのPyTorch tensor: {file_name}, サイズ: {tensor.size()}")


In [5]:
import torch
import glob

fol_name = "231021_iter1"

# pred_varを全て読み込み、torch.meanとtorch.maxの平均を計算する
pred_var_path = sorted(glob.glob(f'data/unlabeled_mask/{fol_name}/pred_var/*'))
mean_box = []
max_box = []

for file in pred_var_path:
    pred_var = torch.load(file)
    mean_box.append(torch.mean(pred_var).item())
    max_box.append(torch.max(pred_var).item())

print(sum(mean_box)/len(mean_box))
print(sum(max_box)/len(max_box))

0.00019613194298671443
0.05264750664102345
